**Tutorial 2**

**Before running this code:** Make sure you have the following Julia packages installed: `InteractiveIsing`, `Plots`, `GLMakie`, `FileIO`.

**Install via Julia's package manager**

```julia
using Pkg
Pkg.add(url="https://github.com/rug-minds/InteractiveIsing.jl")
Pkg.add("Plots")
Pkg.add("GLMakie")
Pkg.add("FileIO")
Pkg.add("CairoMakie")
```

**What this tutorial covers:**
This tutorial demonstrates advanced features of InteractiveIsing, including:
- Custom weight functions for different coupling patterns
- Spin glass systems with random and structured disorder

**Prerequisites:**
Please ensure you have completed Tutorial 1 to understand the basic concepts of Ising simulations and hysteresis loops.

In [7]:
## Utility functions for experiments

function newmakie(makietype, args...)
    f = makietype(args...)
    scr = GLMakie.Screen()
    display(scr, f)
    f
end
# Random partition of n into 'parts' segments (returns cumulative sizes)
function random_partition(n, parts)
    if parts == 0
        return [n]
    end
    cuts = sort(rand(1:n-1, parts-1))
    partition = [cuts[1]; diff(cuts); n - cuts[end]]
    return cumsum(partition)
end

# Uniform partition of n into 'parts' nearly equal segments (returns cumulative sizes)
function uniform_partition(n::Integer, parts::Integer)
    if parts == 0
        return [n]
    end
    base = div(n, parts)       # base size of each part
    rem = mod(n, parts)        # remainder to distribute
    partition = [i <= rem ? base + 1 : base for i in 1:parts]
    return cumsum(partition)
end

# Generate random numbers around x with range ±y
function randAround(x::Number, y::Number, dims::Int...)
    x = float(x)
    y = float(y)
    return x .+ y .* (2 .* rand(dims...) .- 1)
end


##################################################################################
### Pulse type: TrianglePulseA (simple four-segment triangular waveform)
struct TrianglePulseA end

function Processes.prepare(::TrianglePulseA, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    first  = LinRange(0, amp, round(Int,rise_point))
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    fourth = LinRange(-amp, 0, round(Int,rise_point))
    pulse = vcat(first, second, third, fourth)
    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        "Wrong length"
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::TrianglePulseA)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


##################################################################################
### Pulse type: PUNDa (no delay time between pulses)
struct PUNDa end

function Processes.prepare(::PUNDa, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    first  = LinRange(0, amp, round(Int,rise_point))
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    fourth = LinRange(-amp, 0, round(Int,rise_point))
    pulse = vcat(first, second, first, second, third, fourth, third, fourth)
    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        println("Pulse is longer than lifetime")
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    all_Es = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::PUNDa)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


##################################################################################
### Pulse type: PUNDb (with delay time between each pulse)
struct PUNDb end

function Processes.prepare(::PUNDb, args)
    (;amp, numpulses, rise_point) = args
    steps = num_calls(args)

    delay  = zeros(Int, rise_point)
    first  = LinRange(0, amp, round(Int,rise_point))
    amp1   = fill(amp, rise_point)
    second = LinRange(amp, 0, round(Int,rise_point))
    third  = LinRange(0, -amp, round(Int,rise_point))
    amp2   = fill(-amp, rise_point)
    fourth = LinRange(-amp, 0, round(Int,rise_point))

    pulse = vcat(delay, first, amp1, second,
                 delay, first, amp1, second,
                 delay, third, amp2, fourth,
                 delay, third, amp2, fourth)

    pulse = repeat(pulse, numpulses)

    if steps < length(pulse)
        println("Pulse is longer than lifetime")
    else
        fix_num = num_calls(args) - length(pulse)
        fix_arr = zeros(Int, fix_num)
        pulse   = vcat(pulse, fix_arr)
    end

    # Predefine storage arrays
    x = Float32[]
    y = Float32[]
    all_Es = Float32[]
    processsizehint!(args, x)
    processsizehint!(args, y)

    return (;pulse, x, y)
end

function (::PUNDb)(args)
    (;pulse, M, x, y, hamiltonian) = args
    pulse_val = pulse[algo_loopidx(args)]
    hamiltonian.b[] = pulse_val
    push!(x, pulse_val)
    push!(y, M[])
end


In [ ]:
# Setup: Load packages and create basic 3D Ising graph
# This follows the same pattern as Tutorial 1

using InteractiveIsing, Plots, GLMakie, FileIO
using InteractiveIsing.Processes
import InteractiveIsing as II

# Initialize a 3D Ising graph with specified dimensions and periodic boundary conditions
xL = 100  # Length in the x-dimension
yL = 100  # Length in the y-dimension
zL = 10   # Length in the z-dimension (thin film geometry)

**Custom Weight Functions**

Weight functions determine the coupling strength between neighboring spins in the Ising model. They are the core mechanism for defining different interaction patterns and building the adjacency matrix.

**How Weight Functions Work in the Code:**

The `@WG` macro creates a `WeightGenerator` that calls your weight function for each pair of dipoles. Here's what actually happens:

1. **For each dipole at position `(x, y, z)`**:
   - Find all neighbors within `NN` range: `(x+dx, y+dy, z+dz)`
   - Call your weight function with parameters: `(dx, dy, dz, x, y, z, ...)`
   - Store the **returned coupling strength** in the adjacency matrix

2. **The weight function returns the actual coupling coefficient `J_ij`**:
   - This value goes directly into the Hamiltonian: `H = -∑ J_ij s_i s_j`
   - Positive values → ferromagnetic coupling (parallel spins favored)
   - Negative values → antiferromagnetic coupling (anti-parallel spins favored)

**Understanding the Distance Scaling `dr2`:**

```julia
dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
return prefactor / dr2  # This J_ij goes into the Hamiltonian
```

**What `dr2` represents:**
- **`(dx, dy, dz)`**: Relative displacement in lattice units (e.g., dx=1 means "one lattice step right")
- **`(rx, ry, rz)`**: **User-defined scaling factors** - you control these!
- **`dr2`**: Your custom distance metric for determining coupling strength

**Key Insight - You Define the Scaling Rule:**

The scaling factors `(rx, ry, rz)` are **entirely up to you**. They let you:
- **Approximate realistic dipole interactions**: Use physical lattice constants  
- **Create anisotropic coupling**: Different strengths along x, y, z directions
- **Model material properties**: Account for crystal structure, defects, etc.
- **Design artificial patterns**: Create specific domain structures

**Examples of Distance Scaling:**
- **Isotropic**: `rx = ry = rz = 1` → uniform spacing in all directions
- **Anisotropic**: `rx = 1, ry = 1, rz = 0.5` → compressed interactions along z-axis  
- **Physical units**: `rx = 3.9Å, ry = 3.9Å, rz = 4.1Å` → real lattice constants
- **Effective scaling**: `rx = 1.2, ry = 0.8, rz = 1.0` → tuned for specific material behavior

**Why Use 1/r² Decay:**

This is a **modeling choice** to approximate how dipole interaction energy decays with distance:
- **Closer dipoles** → smaller `dr2` → larger coupling `|J_ij|`
- **Farther dipoles** → larger `dr2` → weaker coupling `|J_ij|`
- **Balance**: Strong enough to capture nearest-neighbor physics, weak enough for computational stability

**The Power is in Your Hands:**
You can use any distance function you want: `1/dr2`, `1/dr2^1.5`, `exp(-dr2)`, or even completely custom rules. The weight function returns the final coupling strength that determines how strongly each pair of dipoles interacts.

In [ ]:
# Weight function variant 1: Pure ferromagnetic coupling
function weightfunc1(dx, dy, dz, rx, ry, rz)
    # Always positive coupling (ferromagnetic)
    dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
    return 1.0 / dr2
end

# Weight function variant 2: Checkerboard antiferromagnetic pattern
function weightfunc2(dx, dy, dz, rx, ry, rz)
    # Alternating signs based on lattice position parity
    dr2 = (rx*dx)^2 + (ry*dy)^2 + (rz*dz)^2
    
    # Checkerboard pattern: alternating ferro/antiferro based on coordinate sum
    if (abs(dx) + abs(dy)) % 2 == 0
        return 1.0 / dr2    # Ferromagnetic coupling
    else
        return -1.0 / dr2   # Antiferromagnetic coupling
    end
end

**Advanced Weight Functions: Spin Glass Systems**

Now we explore complex interaction patterns that create spin glass behavior and domain structures by using **spatially-varying coupling parameters**.

**Understanding Weight Function Parameters:**

Weight functions can take many parameters. The most common pattern is:
```julia
function my_weight_func(dx, dy, dz, x, y, z, additional_params...)
```

**Parameter Roles:**
- **`(dx, dy, dz)`**: **Relative displacement** to the neighbor (e.g., dx=1 means "one step right")
- **`(x, y, z)`**: **Absolute coordinates** of the central dipole in the lattice
- **`additional_params...`**: **Your custom parameters** (domain boundaries, random coefficients, etc.)

**Creating Spatial Disorder:**

The key insight is that you can make the coupling strength **location-dependent**:

1. **Divide the lattice into domains** using partition functions:
   ```julia
   x_n = random_partition(xL, 20)  # 20 random domains along x
   y_n = uniform_partition(yL, 10) # 10 uniform domains along y
   ```

2. **Assign domain-specific properties**:
   ```julia
   coef_x = randAround(1, 0.2, 20)  # Random scaling around 1.0 ± 0.2
   coef_y = randAround(1, 0.2, 10)  # Different for each domain
   ```

3. **Use absolute coordinates `(x, y, z)` to determine which domain**:
   ```julia
   ix = findfirst(v -> x ≤ v, x_n)  # Which x-domain contains this dipole?
   iy = findfirst(v -> y ≤ v, y_n)  # Which y-domain contains this dipole?
   ```

4. **Apply domain-specific scaling to the distance**:
   ```julia
   dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2  # Domain-dependent distance
   ```

**Physical Interpretation:**

This creates **realistic material disorder**:
- **Different domains** have different "effective lattice constants" 
- **Strain/stress variations**: Some regions compressed (`coef < 1`), others stretched (`coef > 1`)
- **Chemical inhomogeneity**: Different local compositions change interaction strength
- **Defects and interfaces**: Domain boundaries create frustration and pinning sites

**The Result: Spin Glass Behavior**

When you combine:
- **Spatial disorder** in coupling strengths (different `coef` values)
- **Competing interactions** (ferromagnetic vs antiferromagnetic patterns)  
- **Long-range connections** (beyond nearest neighbors)

You get **frustrated energy landscapes** where:
- No single configuration can satisfy all interactions
- Multiple local energy minima exist
- System shows complex, history-dependent switching behavior

**Practical Example: Domain-Based Coupling**

Let's see how `weightfunc_glass_anti()` actually works:

```julia
function weightfunc_glass_anti(dx, dy, dz, x, y, z, x_n, y_n, z_n, coef_x, coef_y, coef_z)
    # Step 1: Determine which domain this dipole belongs to
    ix = findfirst(v -> x ≤ v, x_n)  # Domain index along x
    iy = findfirst(v -> y ≤ v, y_n)  # Domain index along y  
    iz = findfirst(v -> z ≤ v, z_n)  # Domain index along z
    
    # Step 2: Use domain-specific scaling coefficients
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 + (coef_z[iz] * dz)^2
    
    # Step 3: Apply interaction pattern (checkerboard antiferromagnetic)
    prefac = (abs(dx) + abs(dy)) % 2 != 0 ? -1 : 1
    
    # Step 4: Return the coupling strength J_ij
    return prefac / dr2
end
```

**What This Achieves:**

1. **Heterogeneous Coupling Strengths**: 
   - Domain A: `coef_x = 1.2` → weaker interactions (`J ∝ 1/1.44`)
   - Domain B: `coef_x = 0.8` → stronger interactions (`J ∝ 1/0.64`)

2. **Realistic Material Properties**:
   - Models regions with different local environments
   - Creates natural disorder without completely random coupling
   - Maintains physical distance-dependence within each domain

3. **Controlled Frustration**:
   - Systematic rather than purely random disorder
   - Allows investigation of specific domain effects
   - Can model real material interfaces and defects

**Key Advantage**: You're not just creating random couplings - you're building **physically motivated spatial patterns** that capture realistic material heterogeneity while maintaining the essential physics of distance-dependent interactions.

In [ ]:
x_n = random_partition(xL, 20)
y_n = random_partition(yL, 20)
z_n = random_partition(zL, 3)
coef_x =randAround(1, 0.2, 20)
coef_y =randAround(1, 0.2, 20)
coef_z =randAround(1, 0.2, 3)
println(x_n)
println(y_n)
println(z_n)
println(coef_x)
println(coef_y)
println(coef_z)


function weightfunc_glass_anti(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x,coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    if (abs(dx) + abs(dy)) % 2 != 0  
        prefac = -1
    else  
        prefac = 1
    end
    return prefac / dr2
end

function weightfunc_glass_ferro(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x,coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    prefac = 1
    return prefac / dr2
end

function weightfunc_AntiWithFerro(dx, dy, dz, x, y, z, x_n, y_n,z_n, coef_x, coef_y, coef_z)
    prefac = 1
    ix = findfirst(v -> x ≤ v, x_n)
    iy = findfirst(v -> y ≤ v, y_n)
    iz = findfirst(v -> z ≤ v, z_n)
    if isnothing(ix) || isnothing(iy)
        error("out of range")
    end
    dr2 = (coef_x[ix] * dx)^2 + (coef_y[iy] * dy)^2 +(coef_z[iz]* dz)^2
    if ix % 2 == 0 && iy %2==0
        prefac=1
    elseif xor(isodd(ix), isodd(iy))
        if (abs(dx) + abs(dy)) % 2 != 0  
            prefac = -1
        else  
            prefac = 1
        end
    end
    return prefac / dr2
end

wg_glass1 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_glass_ferro(dx,dy,dz,x,y,z,x_n, y_n, z_n, coef_x,coef_y,coef_z)" NN = (2,2,2)
wg_glass2 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_glass_anti(dx,dy,dz,x,y,z, x_n, y_n, z_n, coef_x,coef_y,coef_z)" NN = (2,2,2)
wg_glass3 = @WG "(dx,dy,dz,x,y,z) -> weightfunc_AntiWithFerro(dx, dy, dz, x, y, z, x_n, y_n, z_n, coef_x, coef_y, coef_z)" NN = (2,2,2)


**Simulation Examples**

Now we'll run three different simulations to demonstrate how different weight functions affect the hysteresis behavior:

1. **Example 1**: Standard ferromagnetic coupling with uniform interactions
2. **Example 2**: Spin glass with antiferromagnetic checkerboard pattern  
3. **Example 3**: Mixed ferro/antiferromagnetic domains

Each simulation uses the same external field protocol (triangular pulse) but different internal coupling patterns. Compare the resulting hysteresis loops to see how the microscopic interactions influence macroscopic behavior.

In [ ]:
# Example 1: normal weight function (ferro)
g = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g.hamiltonian = Ising(g) + DepolField(g, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g.hamiltonian = sethomogenousparam(g.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.

homogeneousself!(g, -1)

### We set Temperature to 1.5
settemp(g,1.5)

### Change nearest neighbours
####################################################################################
wg1 = @WG "(dx,dy,dz,x,y,z) -> weightfunc1(dx,dy,dz,1,1,1)" NN = (2,2,2)
####################################################################################
genAdj!(g[1], wg1)


fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)
### estimate time
est_remaining(process(g))

# Wait until it is done
args = process(g) |> fetch # If you want to close ctr+c
# args = process(g) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

# w1=newmakie(lines, Pr, EnergyG)
w2=newmakie(lines, voltage, Pr)
w3=newmakie(lines,Pr)


**Results Analysis - Example 1 vs Example 2**

Compare the hysteresis loops from the two simulations:

- **Example 1 (ferro)**: Should show clean, symmetric loops with sharp switching
- **Example 2 (spin glass anti)**: Expect more rounded loops, possible pinching, and asymmetric behavior

The spin glass system creates **competing interactions** that lead to:
- Frustrated energy landscapes
- Multiple metastable states  
- History-dependent switching behavior
- Reduced coercive field uniformity

In [ ]:
# Example 2：spin glass anti alignment
g_glass_anti = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g_glass_anti)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g_glass_anti.hamiltonian = Ising(g_glass_anti) + DepolField(g_glass_anti, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g_glass_anti.hamiltonian = sethomogenousparam(g_glass_anti.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.
homogeneousself!(g_glass_anti, -1)

### We set Temperature to 1.5
settemp(g_glass_anti,1.5)


genAdj!(g_glass_anti[1], wg_glass2)

fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g_glass_anti, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)

### estimate time
est_remaining(process(g_glass_anti))

# Wait until it is done
args = process(g_glass_anti) |> fetch # If you want to close ctr+c
# args = process(g_glass_anti) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

# w1=newmakie(lines, Pr, EnergyG)
w_anti_2=newmakie(lines, voltage, Pr)
w_anti_3=newmakie(lines,Pr)


**Example 3: Mixed Ferro/Antiferromagnetic Domains**

This simulation demonstrates the most complex case where:
- **Even-indexed domains**: Pure ferromagnetic behavior
- **Odd-indexed domains**: Mixed ferro/antiferromagnetic patterns based on XOR logic
- **Domain boundaries**: Create additional frustration and pinning sites

Expected behavior:
- Intermediate loop characteristics between Examples 1 and 2
- Possible multi-step switching due to domain heterogeneity
- Enhanced coercive field distribution

In [ ]:
# Example 3：spin glass with mixed interactions
g_GlassMix = IsingGraph(xL, yL, zL, type = Continuous(), periodic = (:x,:y))

# Set visual marker size for clarity
II.makie_markersize[] = 0.3

# Launch interactive visualization (will remain idle until we create a process)
interface(g_GlassMix)

# Set depolarization (DP) field:
# Smaller c => stronger depolarization penalty (suppresses net polarization, can pinch loops)
# Larger  c => weaker depolarization (permits larger remanent polarization)
# Here: c=60000 (weak DP) for a clear baseline hysteresis loop.
g_GlassMix.hamiltonian = Ising(g_GlassMix) + DepolField(g_GlassMix, c=60000, left_layers=1, right_layers=1)

# Make external field parameter :b homogeneous & mutable so pulse routines can update hamiltonian.b[]
g_GlassMix.hamiltonian = sethomogenousparam(g_GlassMix.hamiltonian, :b)

# Apply a moderate negative self-energy bias.
# This favors polarized states (ferroelectric tendency),
# making flips toward zero polarization less likely
# and stabilizing domain alignment.
homogeneousself!(g_GlassMix, -1)

### We set Temperature to 1.5
settemp(g_GlassMix,1.5)


genAdj!(g_GlassMix[1], wg_glass3)

fullsweep = xL*yL*zL
Time_fctr = 0.2
SpeedRate = Int(Time_fctr*fullsweep)
risepoint=500
Amptitude =20
# risepoint = round(Int, Amptitude/0.01)

#### Run with TrianlePulseA
PulseN = 2
Pulsetime = (PulseN * 4 + 2) * risepoint * SpeedRate
compalgo = CompositeAlgorithm((Metropolis, TrianglePulseA), (1, SpeedRate))
createProcess(g_GlassMix, compalgo, lifetime =Pulsetime, amp = Amptitude, numpulses = PulseN, rise_point=risepoint)

### estimate time
est_remaining(process(g_GlassMix))

# Wait until it is done
args = process(g_GlassMix) |> fetch # If you want to close ctr+c
# args = process(g_GlassMix) |> getargs
# EnergyG= args.all_Es;
voltage= args.x
Pr= args.y;

w_Mix_2=newmakie(lines, voltage, Pr)
w_Mix_3=newmakie(lines,Pr)